In [ ]:
!pip install contractions chardet rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=724a7c5ca2f6dd200aba328693f5761bf61256334d068c7d88990755625f26ec
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


# Importing Required Libraries

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import unicodedata
import chardet
import string
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup


# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
dataset=pd.read_excel('366_ARPs_for_extracting_Issue_Solution_Pairs.xlsx')

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Post Preprocessing

In [ ]:
# Function to remove HTML tags and replace links, images, code snippets, and tables with placeholders
def clean_html(text):
    if not isinstance(text, str):
        return text

    soup = BeautifulSoup(text, "html.parser")

    # Replace all links with '[external-link]'
    for a in soup.find_all('a'):
        a.replace_with('[external-link]')

    # Replace all images with '[figure]'
    for img in soup.find_all('img'):
        img.replace_with('[figure]')

    # Replace all code blocks with '[code-snippet]'
    for code in soup.find_all('code'):
        code.replace_with('[code-snippet]')

    # Replace all tables with '[table]'
    for table in soup.find_all('table'):
        table.replace_with('[table]')

    # Get the text without any remaining HTML tags
    clean_text = soup.get_text()

    return clean_text

dataset = pd.read_excel('366_ARPs_for_extracting_Issue_Solution_Pairs.xlsx')

# Apply the function to 'Question_body' and 'Answer_body' columns
dataset['Question_body_cleaned'] = dataset['Question_body'].apply(clean_html)
dataset['Answer_body_cleaned'] = dataset['Answer_body'].apply(clean_html)

In [ ]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    sentences = sent_tokenize(text)

    stop_words = set(ENGLISH_STOP_WORDS)

    punctuation = set(string.punctuation)
    extra_special_characters = set(["''", '``', '##', '>>', '<<', 'e', 'g', 'eg', 'cant', 'cannot', 'isnt', 'would', 'could', 'doesnt', 'hasnt', 'thanks', '-', ')', '\n'])
    special_characters = extra_special_characters.union(punctuation)

    processed_sentences = []

    for sentence in sentences:
        cleaned_words = re.findall(r'\b\w+\b', sentence.lower())
        cleaned_words = [word for word in cleaned_words if word not in stop_words]
        cleaned_words = [word for word in cleaned_words if word not in special_characters]
        cleaned_words = [word for word in cleaned_words if not word.isdigit()]
        cleaned_words = [word for word in cleaned_words if word.isalpha()]

        lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]

        processed_sentences.append(cleaned_words)

    return sentences, processed_sentences

In [ ]:
# Preprocess only the Question_body_cleaned part
dataset['processed_question'] = dataset['Question_body_cleaned'].apply(lambda x: preprocess_text(x)[1])
dataset['processed_answer'] = dataset['Answer_body_cleaned'].apply(lambda x: preprocess_text(x)[1])

# Display the first five rows
dataset[['processed_question', 'processed_answer']].head()


,processed_question,processed_answer
0,"[[need, help, architecture, pattern, use, nest...","[[talking, strictly, cqrs, pattern, designing,..."
1,"[[file, structure, processes, fromjson, tojson...","[[purpose, clean, architecture, fromjson, tojs..."
2,"[[building, c, net, mvc, web, application, all...","[[easy, reliable, way, splitting, task, multip..."
3,"[[mobile, application, scale, white, label, de...","[[simple, answer, question], [think, tailwind,..."
4,"[[m, trying, properly, design, application, ac...","[[determining, source, information, business, ..."


# Build term-document matrix function

In [ ]:
# Build similarity matrix based on TF-IDF and LSA
def build_similarity_matrix(processed_sentences):
    sentences = [" ".join(sentence) for sentence in processed_sentences]

    # Compute the TF-IDF matrix
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    # Determine the number of components based on the number of features
    num_features = tfidf_matrix.shape[1]
    n_components = min(num_features // 2, 100)

    # Apply LSA (Latent Semantic Analysis). LSA uses Singular Value Decomposition (SVD)
    lsa = TruncatedSVD(n_components=n_components)
    lsa_matrix = lsa.fit_transform(tfidf_matrix)

    # Compute the cosine similarity matrix based on the LSA-transformed matrix
    similarity_matrix = cosine_similarity(lsa_matrix)

    return similarity_matrix

# Build similarity matrices for both processed questions and answers
dataset['question_similarity_matrix'] = dataset['processed_question'].apply(lambda x: build_similarity_matrix(x))
dataset['answer_similarity_matrix'] = dataset['processed_answer'].apply(lambda x: build_similarity_matrix(x))

# Preview the results
#print(dataset[['question_similarity_matrix', 'answer_similarity_matrix']].head())

# Sentences Ranking

In [ ]:
# Rank sentences based on their global importance
def rank_sentences(similarity_matrix):
    scores = np.mean(similarity_matrix, axis=1)
    return scores

# Extracting Issue-Solution Pairs

In [ ]:
# Generate summary function
from nltk.tokenize import sent_tokenize

def generate_summary(original_text, similarity_matrix, num_sentences=6):
    # Tokenize the original text into sentences
    original_sentences = sent_tokenize(original_text)

    ranked_sentence_indices = np.argsort(np.mean(similarity_matrix, axis=1))[::-1]
    summary = []
    selected_indices = []

    for i in range(len(ranked_sentence_indices)):
        if len(summary) >= num_sentences:
            break

        candidate_index = ranked_sentence_indices[i]
        candidate_sentence = original_sentences[candidate_index]

        # Check similarity with already selected sentences
        is_redundant = False
        for selected_index in selected_indices:
            similarity = cosine_similarity([similarity_matrix[candidate_index]], [similarity_matrix[selected_index]])[0][0]
            if similarity > 0.6:  # Adjust threshold as needed
                is_redundant = True
                break

        if not is_redundant:
            summary.append(candidate_sentence)
            selected_indices.append(candidate_index)

    return " ".join(summary)

In [ ]:
# Generate summaries/issue-solutions pairs based on ranked sentences
dataset['Issue_Extracted'] = dataset.apply(lambda x: generate_summary(x['Question_body_cleaned'], x['question_similarity_matrix']), axis=1)
dataset['Solution_Extracted'] = dataset.apply(lambda x: generate_summary(x['Answer_body_cleaned'], x['answer_similarity_matrix']), axis=1)


In [ ]:
# Display the summaries/issue-solutions pairs
summaries = dataset[['Question_title', 'Issue_Extracted', 'Solution_Extracted']]
summaries.head()

,Question_title,Issue_Extracted,Solution_Extracted
0,Separation of Students and Users in NestJS Mic...,"Considering this, creating users, contacts, an...",If something cannot be done with one repositor...
1,Flutter Clean Architecture,I created entity class on business layer and a...,"For the purpose of clean architecture, the fro..."
2,Correct .NET Architecture for long running asy...,If they execute it immediately the web page sh...,Cancellation of the operation can also be mark...
3,Architecture for white-label mobile apps with ...,It becomes a nightmare to move features across...,Now imagine if you have a pure UI layer that g...
4,Implementing Data Source Selection Logic in Cl...,"Then, as I understand it, there are several ap...",Determining the source of the information is b...


In [ ]:
# Save the summarized data to a new Excel file
file_path = '/content/LSA_Extracted_Issue_Solution.xlsx'

dataset.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
# Verify that the file has been saved correctly
#saved_data = pd.read_excel(file_path)
#print(saved_data[['Question_title', 'Issue_Extracted', 'Solution_Extracted']].head())

# Evaluate the extracted issue-solution pairs using Precsion, Recall, and F1

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd


def evaluate_summaries_at_sentence_level(df, ref_col, gen_col):
    precision_list = []
    recall_list = []
    f1_list = []

    for index, row in df.iterrows():
        ref_summary = row[ref_col]
        gen_summary = row[gen_col]

        if pd.isna(ref_summary) or pd.isna(gen_summary):
            continue

        ref_sentences = nltk.sent_tokenize(ref_summary)
        gen_sentences = nltk.sent_tokenize(gen_summary)

        ref_sentences_set = set(ref_sentences)
        gen_sentences_set = set(gen_sentences)

        precision = len(ref_sentences_set & gen_sentences_set) / len(gen_sentences_set) if gen_sentences_set else 0
        recall = len(ref_sentences_set & gen_sentences_set) / len(ref_sentences_set) if ref_sentences_set else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    metrics_df = pd.DataFrame({
        'precision': precision_list,
        'recall': recall_list,
        'f1': f1_list
    })

    return metrics_df

# Load your DataFrame
df = pd.read_excel('LSA_Extracted_Issue_Solution.xlsx')

question_metrics_df = evaluate_summaries_at_sentence_level(df, 'Ground_truth_Issue_Labeled', 'Issue_Extracted')
answer_metrics_df = evaluate_summaries_at_sentence_level(df, 'Ground_truth_Solution_Labeled', 'Solution_Extracted')

# Separate evaluation for question and answer summaries, retaining individual columns for comparison
question_metrics_df.columns = [f'Question_{col}' for col in question_metrics_df.columns]
answer_metrics_df.columns = [f'Answer_{col}' for col in answer_metrics_df.columns]

# Combine question and answer results into a single DataFrame
combined_metrics_df = pd.concat([question_metrics_df, answer_metrics_df], axis=1)

# Compute overall Precision, Recall, F1 scores (separately for questions and answers)
mean_question_metrics = question_metrics_df.mean()
mean_answer_metrics = answer_metrics_df.mean()

print("\nMean Precision, Recall, F1 Scores for \033[31mQuestions\033[0m:")
print(mean_question_metrics)

print("\nMean Precision, Recall, F1 Scores for \033[31mAnswers\033[0m:")
print(mean_answer_metrics)



Mean Precision, Recall, F1 Scores for Questions:
Question_precision    0.573270
Question_recall       0.539787
Question_f1           0.552927
dtype: float64

Mean Precision, Recall, F1 Scores for Answers:
Answer_precision    0.554234
Answer_recall       0.516716
Answer_f1           0.530765
dtype: float64
